In [ ]:
# NOT WORKING

import os
import base64
import io
import requests
import json
from dotenv import load_dotenv
from PIL import Image  # pip install Pillow

# --- SETUP ---
load_dotenv()
api_key = os.getenv("NVIDIA_API_KEY_2")

# Use 'r' for raw string to fix backslash errors
INPUT_IMAGE = r"C:\Data\ai_engineer_course\01_workflow\images\ka.jpg" 
OUTPUT_FILENAME = "output_model.glb"
INVOKE_URL = "https://ai.api.nvidia.com/v1/genai/microsoft/trellis"

if not api_key:
    raise ValueError("NVIDIA_API_KEY not found. Check your .env file.")

# --- HELPER FUNCTION ---
def process_and_encode_image(image_path):
    """
    Reads an image, resizes it to max 1024x1024 (safe for API),
    converts it to PNG, and returns the base64 string.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found at: {image_path}")

    # Open the image
    with Image.open(image_path) as img:
        # Convert to RGB (fixes issues with some JPEGs or transparent PNGs)
        img = img.convert("RGBA")
        
        # Resize if too big (max 1024x1024 is usually safe for Trellis)
        img.thumbnail((1024, 1024)) 
        
        # Save to a memory buffer as PNG
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        
        # Encode to Base64
        base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
        
        # NVIDIA requires this specific prefix
        return f"data:image/png;base64,{base64_image}"

# --- MAIN EXECUTION ---
try:
    print(f"Processing and resizing image: {INPUT_IMAGE}...")
    image_b64 = process_and_encode_image(INPUT_IMAGE)
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Accept": "application/json",
    }

    payload = {
        "image": image_b64,
        "slat_cfg_scale": 3,
        "ss_cfg_scale": 7.5,
        "slat_sampling_steps": 25,
        "ss_sampling_steps": 25,
        "seed": 0
    }

    print("Sending request to NVIDIA API...")
    response = requests.post(INVOKE_URL, headers=headers, json=payload)

    response.raise_for_status()
    response_body = response.json()

    # Save the result
    if 'generated_model' in response_body:
        glb_data = base64.b64decode(response_body['generated_model'])
        with open(OUTPUT_FILENAME, "wb") as f:
            f.write(glb_data)
        print(f"✅ Success! 3D model saved to: {os.path.abspath(OUTPUT_FILENAME)}")
    else:
        print("⚠️ Response format unexpected. Printing raw JSON:")
        print(json.dumps(response_body, indent=2))

except requests.exceptions.HTTPError as err:
    print(f"❌ HTTP Error: {err}")
    print(f"Details: {response.text}")
except Exception as e:
    print(f"❌ Error: {e}")